<a href="https://colab.research.google.com/github/thaopham03/evaluating_fan_effects_in_large_language_models/blob/main/fan_effects_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install external libraries:

!pip install git+https://github.com/JesseTNRoberts/minicons_modded
!pip install git+https://github.com/JesseTNRoberts/PopulationLM
!pip install accelerate transformers

  Cloning https://github.com/JesseTNRoberts/minicons_modded to /tmp/pip-req-build-r4bpoq81
  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/minicons_modded /tmp/pip-req-build-r4bpoq81
  Resolved https://github.com/JesseTNRoberts/minicons_modded to commit 2aa9e4e05fb5c2af99f10dc75cc1e2968b5eaceb
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/JesseTNRoberts/PopulationLM to /tmp/pip-req-build-nucqf1o4
  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/PopulationLM /tmp/pip-req-build-nucqf1o4
  Resolved https://github.com/JesseTNRoberts/PopulationLM to commit e7adde2326e644d7c94d2c7086d9561f8aefb977
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import argparse
import csv
from tqdm import tqdm
import numpy as np
import random
import accelerate

import transformers
import os
import shutil
import glob

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM
from torch.utils.data import DataLoader

from minicons import scorer
import PopulationLM as pop

# from google.colab import userdata
# token = userdata.get('HF_TOKEN')

import gc

from google.colab import drive
drive.mount('/content/drive')

committee_size = 50
add_dropout_layers = True


inpath = '/content/drive/MyDrive/IGOG-Alignment-Exp/Experiments/fan-effects-exp/generated_birds_prompts.csv'

cachepath = '/content/drive/MyDrive/IGOG-Alignment-Exp/cache/'


# make results dir: ../IGOG-Alignment-Exp/Experiments/fan-effects-exp/results/ration/model_name.csv
components = inpath.split("/")
data_dir = "/".join(components[0:-1])
dataset_name = components[-1].split(".")[0]
results_dir = f"{data_dir}/results/rational"

dataset = []
with open(inpath, "r") as f:
    reader = csv.DictReader(f)
    column_names = reader.fieldnames
    for row in reader:
        dataset.append(list(row.values()))

batch_size = 54
device = 'cuda'
stimuli_loader = DataLoader(dataset, batch_size = batch_size, num_workers=0)

models = [
    ['bert-base-uncased',  'mlm'],
    # ['bert-large-uncased', 'mlm'],
    # ['distilbert-base-uncased', 'mlm'],
    # ['roberta-base', 'mlm'],
    # ['roberta-large', 'mlm'],
    # ["google/electra-large-generator", 'mlm'],
    # ['FacebookAI/xlm-roberta-large', 'mlm'],
    # ['distilgpt2', 'incremental'],
    # ['gpt2-medium',  'incremental'],
    # ['gpt2',  'incremental'],
    # ['openai-gpt',  'incremental'],
    # ["daryl149/llama-2-7b-hf", 'incremental'],
    # ["mistralai/Mistral-7B-v0.1",  'incremental'],
    # ["google/gemma-7b",  'incremental'],
    # ['PKU-Alignment/alpaca-7b-reproduced', 'incremental'],
    # ['openlm-research/open_llama_13b',  'incremental'],
    # ['upstage/SOLAR-10.7B-v1.0',  'incremental'],
    # ['microsoft/phi-2',  'incremental'],
    # ['meta-llama/Llama-2-13b-hf',  'incremental'],
]

use_population = True
local_only = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for model in models:
    model_name = model[0].replace("/", "_")
    if os.path.exists(results_dir + f"/{model_name}.csv"):
        continue

    print(model[0])

    try:
        if model[1] == "mlm":
            # try:
                transformer = scorer.MaskedLMScorer(model[0], device, token=None, cache_dir=cachepath,
                                                    local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto")
            # except ValueError as e:
            #     if "device_map" in str(e):
            #         transformer = scorer.MaskedLMScorer(model[0], device, token=None, cache_dir=cachepath,
            #                                             local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16)
            #     else:
            #         raise e
        elif model[1] == "incremental":
            # try:
                transformer = scorer.IncrementalLMScorer(model[0], device, token=None, cache_dir=cachepath,
                                                    local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto")
        #     except ValueError as e:
        #         if "device_map" in str(e):
        #             transformer = scorer.IncrementalLMScorer(model[0], device, token=None, cache_dir=cachepath,
        #                                                     local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16)
        #         else:
        #             raise e
        # else:
        #     raise ValueError("Invalid model type")

    except Exception as e:
        print(f"Error loading model {model[0]}: {e}")
        continue

    num_params = [sum(p.numel() for p in transformer.model.parameters())] * len(dataset)

    for val in [0.1]:
        if use_population:
            pop.DropoutUtils.convert_dropouts(transformer.model)
            pop.DropoutUtils.activate_mc_dropout(transformer.model, activate=True, random=val)

        results = []
        control_results = []
        conclusion_only = []

        call_me = lambda prefixes, queries: transformer.conditional_score(prefixes, queries, reduction=lambda x: (x.sum(0).item(), x.mean(0).item(), x.tolist()))

        for batch in stimuli_loader:
            premise = list(batch[0])
            conclusion = list(batch[1])

            if use_population:
                population = pop.generate_dropout_population(transformer.model, lambda: call_me(premise, conclusion), committee_size=committee_size)

            print(len(premise), len(conclusion))

            if use_population:
                outs = [item for item in tqdm(pop.call_function_with_population(transformer.model, population, lambda: call_me(premise, conclusion)),
                                              total=committee_size)]
            else:
                outs = [call_me(premise, conclusion)]

            transposed_outs = [[row[i] for row in outs] for i in range(len(outs[0]))]
            priming_scores = [score for score in transposed_outs]
            results.extend(priming_scores)

        data_out = list(zip(*dataset))
        new_col_names = column_names

        data_out.append(results)
        new_col_names += ["score (sum, mean, [list)"]

        data_out.append(num_params)
        data_out.append([model_name] * len(results))
        new_col_names += ["params", "model"]

        with open(results_dir + f"/{model_name}"+".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(new_col_names)
            writer.writerows(list(zip(*data_out)))
            f.flush()

    gc.collect()
    torch.cuda.empty_cache()
